# Getting data

We will get data about yellow taxi trips in New York.

Datasets can be downloaded from:
- yellow taxi 2018: [yellow taxi 2018](https://data.cityofnewyork.us/Transportation/2018-Yellow-Taxi-Trip-Data/t29m-gskq)
- yellow taxi 2019: [yellow taxi 2019](https://data.cityofnewyork.us/Transportation/2019-Yellow-Taxi-Trip-Data/2upf-qytp)

As datasets are large we will get them in chunks: 1.000.000 rows each. We will use [``sodapy``](https://pypi.org/project/sodapy/) package.

In [1]:
import pandas as pd
from sodapy import Socrata
import time
import os

### Setting variables

Instructions about getting data can be found [here](https://dev.socrata.com/foundry/data.cityofnewyork.us/t29m-gskq).

First, one should create a TOKEN (described in the page above).

In [2]:
TOKEN = '<YOUR TOKEN>'
DOMAIN = 'data.cityofnewyork.us'
TIMEOUT = 3600*3 # query timeout in seconds

In [4]:
# We have to get two datasets. 
# DATASET is a name of dataset and custom_name is how dataset will be named in temporary directory.

# DATASET = 't29m-gskq'; custom_name = 'yellow_taxi_2018'
DATASET = '2upf-qytp'; custom_name = 'yellow_taxi_2019'

In [5]:
TMP_PATH = 'data_tmp' # temporary path for data - for this directory bunches of data will be downloaded.
PATH = 'data' # final path for data

### Creating ``Socrata`` client

First, we have to create `socrata` client.

In [6]:
client = Socrata(DOMAIN, TOKEN, timeout=TIMEOUT)

### Dataset size

We can check dataset size (number of rows).

In [19]:
query = """
select count(*)
"""

row_number = client.get(DATASET, query = query)

In [20]:
row_number = int(row_number[0]['count'])
row_number

84399019

### Getting a data

First let's create some auxiliary functions.

In [21]:
def time_duration(t0, t1):
    """ Function for measuring time ellapsed between t0 and t1 """
    return round((t1-t0)/60, 2)



def get_data(client, dataset, offset, limit, order='tpep_pickup_datetime'):
    """ Function for getting data """
    return client.get(dataset, offset=offset, limit=limit, order='tpep_pickup_datetime')

Now we will get data in chunks and save in temporary directory.

In [22]:
# Initial offset and limit
offset=0
limit=int(1e6)

In [25]:
# Do untile offset is greater than number of rows
T0 = time.time()
while True:
    try:
        t0 = time.time()
        tmp = get_data(client, DATASET, offset, limit)
        results_df = pd.DataFrame.from_records(tmp)
        results_df.to_csv(f'{TMP_PATH}/{custom_name}_{offset}.csv', index=False)
        t1 = time.time()
        print(f'offset: {offset} in {time_duration(t0, t1)} min.')
        print(f'Total time ellapsed: {time_duration(T0, t1)} min.')
    except Exception as ex:
        print("Exception! \n{0}".format(ex))

    offset += limit
    if offset > row_number:
        break
    

offset: 0 in 17.42 min.
Total time ellapsed: 17.42 min.
Exception! 
Expecting value: line 836315 column 2 (char 336576582)
offset: 2000000 in 11.9 min.
Total time ellapsed: 40.6 min.
offset: 3000000 in 10.12 min.
Total time ellapsed: 50.72 min.
offset: 4000000 in 8.32 min.
Total time ellapsed: 59.04 min.
offset: 5000000 in 7.05 min.
Total time ellapsed: 66.09 min.
offset: 6000000 in 6.0 min.
Total time ellapsed: 72.09 min.
offset: 7000000 in 5.87 min.
Total time ellapsed: 77.96 min.
offset: 8000000 in 5.06 min.
Total time ellapsed: 83.03 min.
offset: 9000000 in 5.2 min.
Total time ellapsed: 88.23 min.
offset: 10000000 in 3.44 min.
Total time ellapsed: 91.67 min.
offset: 11000000 in 3.21 min.
Total time ellapsed: 94.88 min.
offset: 12000000 in 2.91 min.
Total time ellapsed: 97.79 min.
offset: 13000000 in 2.88 min.
Total time ellapsed: 100.68 min.
offset: 14000000 in 2.75 min.
Total time ellapsed: 103.43 min.
offset: 15000000 in 2.15 min.
Total time ellapsed: 105.58 min.
offset: 16000000

Some parts of datates were not loaded. We will not trying to load them again due to time limitations.

## Concat all data and save

Now we can concatenate all datasets.

Due to size of a datasets we will load only 5.000 records from each file.

In [26]:
directory = os.path.join(f'{TMP_PATH}/')

In [28]:
data = pd.DataFrame()
for root,dirs,files in os.walk(directory):
    for file in files:
        tmp = pd.read_csv(root+file, nrows=5000)
        data = pd.concat([data, tmp])

In [29]:
data.reset_index(drop=True, inplace=True)

In [30]:
data.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2.0,2001-01-01T00:01:48.000,2001-01-01T00:15:47.000,1.0,1.35,1.0,N,43,170,2.0,9.0,0.0,0.5,0.0,0.0,0.3,9.8,NaN
1,2.0,2001-01-01T00:01:48.000,2001-01-01T00:15:47.000,1.0,1.35,1.0,N,43,170,2.0,9.0,0.0,0.5,0.0,0.0,0.3,9.8,NaN
2,2.0,2001-01-01T00:02:26.000,2001-01-01T00:04:49.000,1.0,0.36,1.0,N,48,48,2.0,3.5,0.0,0.5,0.0,0.0,0.3,4.3,NaN
3,2.0,2001-01-01T00:02:26.000,2001-01-01T00:04:49.000,1.0,0.36,1.0,N,48,48,2.0,3.5,0.0,0.5,0.0,0.0,0.3,4.3,NaN
4,2.0,2001-01-01T00:05:12.000,2001-01-01T00:09:59.000,1.0,0.62,1.0,N,140,229,2.0,5.0,0.0,0.5,0.0,0.0,0.3,5.8,NaN


In [31]:
len(data)

920000

In [33]:
data.to_csv(f'{PATH}/yelllow_taxi_data.csv', index=False)

In [34]:
# client stop
client.close()